# Creating a Funding Transaction

In this section, we’ll create a Lightning channel funding transaction from scratch using Python. We’ll walk through each part of the transaction—how it’s constructed, signed, and how peers exchange messages to share the necessary information to make it happen. We'll test everything using Bitcoin Core in regtest mode.

## Prerequisite knowledge
### For all notebooks
- A high level understanding of the bitcoin. e.g. [Mastering Bitcoin](https://github.com/bitcoinbook/bitcoinbook), in particular [Chapter 6](https://github.com/bitcoinbook/bitcoinbook/blob/develop/ch06.asciidoc).
- A conceptual understanding of [hash functions](https://www.thesslstore.com/blog/what-is-a-hash-function-in-cryptography-a-beginners-guide).
- [Hexadecimal notation](https://inst.eecs.berkeley.edu/~cs61bl/r//cur/bits/decimal-binary-hex.html?topic=lab28.topic&step=2&course=) and [endianness](https://www.freecodecamp.org/news/what-is-endianness-big-endian-vs-little-endian/).
- A high level understanding of the lightning e.g. [Mastering Lightning Network](https://github.com/lnbook/lnbook), in particular [Chapter7](https://github.com/lnbook/lnbook/blob/develop/07_payment_channels.asciidoc), [Chapter 8](https://github.com/lnbook/lnbook/blob/develop/08_routing_htlcs.asciidoc) and [Chapter 9](https://github.com/lnbook/lnbook/blob/develop/09_channel_operation.asciidoc).

### Specific to this notebook:
- SHA256, HASH256, HASH160 - '[Hash Functions chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/hash-functions.ipynb)'
- Bech32 addresses - '[Addresses chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Addresses.ipynb)'
- Bitcoin Script basics - '[Bitcoin Script chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Bitcoin%20Script.ipynb)'
- Lightning Network BOLT #2: '[Peer Protocol for Channel Management](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#channel-establishment-v1)'
- Lightning Network Bolt #3: '[Funding Transaction Output](https://github.com/lightning/bolts/blob/master/03-transactions.md#funding-transaction-output)'

## Channel Establishment v1

The Basis of Lightning Technology ([BOLT](https://github.com/lightning/bolts/blob/master/00-introduction.md)) defines two pathways to create a channel, lets focus on the legacy one because most of the channels are created using it.

    +-----------+                              +---------+
    |           |--(1)---  open_channel  ----->|         |
    |           |<-(2)--  accept_channel  -----|         |
    |           |                              |         |
    |   Alice   |--(3)--  funding_created  --->|   Bob   |
    |           |<-(4)--  funding_signed  -----|         |
    |           |                              |         |
    |           |--(5)---  channel_ready  ---->|         |
    |           |<-(6)---  channel_ready  -----|         |
    +-----------+                              +---------+

    - where node Alice is 'funder' and node Bob is 'fundee'

Alice sends the [`open_channel`](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#the-open_channel-message) message to Bob. This message defines several operational parameters for the channel. Here, we’ll focus on the ones that are relevant to constructing the funding transaction:
* chain_hash - Identifies the specific blockchain where the channel will be opened.
* funding_satoshis - The amount of satoshis the sender is contributing to the channel.
* funding_pubkey - The public key used in the 2-of-2 multisig script of the funding transaction output.

If Bob doesn’t agree with the parameters sent in the `open_channel` message, he sends an [`error_message`](https://github.com/lightning/bolts/blob/master/01-messaging.md#the-error-and-warning-messages) back to Alice. Otherwise, he replies with an [`accept_channel`](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#the-accept_channel-message) message, which includes some parameters from his side. Here, we’ll focus on the ones relevant to the funding transaction:
* funding_pubkey - The public key used in the 2-of-2 multisig script that locks the output of the funding transaction.

## The Funding Transaction

The funding transaction input can consist of one or more UTXOs from Alice (the funder).The outputs include a change output back to Alice and a P2WSH (Pay-to-Witness-Script-Hash)<sup>[BIP141](https://github.com/bitcoin/bips/blob/master/bip-0141.mediawiki#witness-program)</sup> output that funds the channel. The order of the public keys in the output script is determined lexicographically: pubkey1 is the lesser of the two funding_pubkey values (in compressed format), and pubkey2 is the greater.

`2 <pubkey1> <pubkey2> 2 OP_CHECKMULTISIG`

## Setup 

### Requirements
For this chapter we'll need the Bitcoin Core. This notebook has been tested with [v28.0](https://github.com/bitcoin/bitcoin/releases/tag/v28.0).

Below, set the paths for:
1. The bitcoin core functional test framework directory.
2. The directory containing lightning-tx-tutorial.

**You'll need to edit these next two lines for your local setup.**

In [1]:
path_to_bitcoin_functional_test = "/home/pins-dev/Projects/bitcoin/test/functional"
path_to_bitcoin_tx_tutorial = "/home/pins-dev/Projects/lightning-tx-tutorial"

import sys

# Add the functional test framework to our PATH
sys.path.insert(0, path_to_bitcoin_functional_test)
from test_framework.test_shell import TestShell

# Add the lightning-tx-tutorial functions to our PATH
sys.path.insert(0, path_to_bitcoin_tx_tutorial)
from functions import *

import json

In [2]:
# run notebook
%run "/home/pins-dev/Projects/lightning-tx-tutorial/Chapter 0 - Key Derivation/The Lightning Node Keys Derivation.ipynb"

Alice Per Commitment Seed 34b581ec20bf2c6cae3d4d4dcbfddc8a3727a1e9a57c55f3520e770607898c06
Bob Per Commitment Seed 89c994b3ddad4698acee71e42d8bcace48eea739caaba371eb110e77663ec56d
Alice Revocation Basepoint Private Key: c17ac3952ca414190074d1e59ea03fbae253196173908dc8b131af6bd2cc8161
Alice Revocation Basepoint Public Key: 03649c4f865bec74b0a186deef4defad51cfdc141443e38074ea05a7835a953a49
Alice HTLC Basepoint Private Key: 763ae49a20e6668c88602c782716dd83ba6c4cc0333b38810e2bcd7b22c871ac
Alice HTLC Basepoint Public Key: 02816fde4150e4dfcac94eff0b821448fb70f57a56148ba2206cd9b2fd0cc20bdf
Alice Payment Basepoint Private Key: 72d8c12971b58076a1f27eb7938ca442f0b210762b23637443ac2e99dac352a6
Alice Payment Basepoint Public Key: 025f892a06124391e2f38ce35d943cdc09f63e203330dbd9cb6113a903e0738458
Alice Delayed Payment Basepoint Private Key: 7cafce00c54e7241894dcc7c3beaca29dd354139fdb6182198d6c5f1063bfe8d
Alice Delayed Payment Basepoint Public Key: 034aa35219136bb238e072341b20a4bf8fb44a83cdb73dd2bd9

### Creating the Inputs of the Channel Funding Transaction

In order for Alice to be the funder of the channel, we’ll first create a P2WPKH UTXO. To do that, we’ll use the keys previously generated for Alice’s node (in the previous notebook), convert them to P2WPKH addresses, and send 0.201 BTC to Alice’s funding address. This transaction will serve as the input to the channel funding transaction.

In [3]:
alice_funding_address = pk_to_p2wpkh(bytes.fromhex(alice_funding_pubkey), network = "regtest")
print("Alice's funding p2wpkh address: " + alice_funding_address)
alice_change_address = pk_to_p2wpkh(bytes.fromhex(alice_change_pubkey), network = "regtest")
print("Alice's change p2wpkh address: " + alice_change_address)

node = setup_testshell()
txid_to_spend, index_to_spend = fund_address(node, alice_funding_address, 0.201)
print(f"txid_to_spent: {txid_to_spend}, {index_to_spend}")

Alice's funding p2wpkh address: bcrt1qt0vq5dj8qa720lfx3hhzyu9ygsng9js3g02lnl
Alice's change p2wpkh address: bcrt1qt6ax8rh20h6zts0dcqmzdy9kte3pds4lrzwl6s
2025-04-30T18:43:42.019000Z TestFramework (INFO): PRNG seed is: 6217689809071432652
2025-04-30T18:43:42.021000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_xagmrbny
txid_to_spent: f85e808553957ebc568a089619a595e1ce121bd012f931024b0bea9015bd11fb, 1


### Creating the Outputs of the Channel Funding Transaction

To create the outputs of the channel funding transaction, we’ll first construct the 2-of-2 multisig redeem script and convert it into a P2WSH address. The other output will be a change address belonging to Alice, who is funding the channel.

In [4]:
# Compare the public keys lexicographically
if alice_funding_pubkey < bob_funding_pubkey:
    print(f"Alice pubkey is lexicographically lesser than Bob pubkey:\n{alice_funding_pubkey} < {bob_funding_pubkey}")
    pubkey1 = alice_funding_pubkey
    pubkey2 = bob_funding_pubkey
elif alice_funding_pubkey > bob_funding_pubkey:
    print(f"Alice pubkey is lexicographically greater than Bob pubkey:\n{alice_funding_pubkey} > {bob_funding_pubkey}")
    pubkey2 = alice_funding_pubkey
    pubkey1 = bob_funding_pubkey
else:
    print("Error: the public keys are equal.")

# Redeem Script: 2 <pubkey1> <pubkey2> 2 OP_CHECKMULTISIG
# "21" is the length of a 33 bytes (compressed) pubkey in hex notation
redeemScript = bytes.fromhex(
    "52"           # requires 2 signatures
    + "21"         # First pubkey
    + pubkey1      
    + "21"         # Second pubkey
    + pubkey2
    + "52"         # of 2 signatures 
    + "ae")        # OP_CHECKMULTISIG

print(f"Redeem Script:\n{redeemScript.hex()}")

channel_address = script_to_p2wsh(redeemScript, "regtest")
print("channel_address: ",channel_address)

# Our outputs
# Output1 is the Alice Change P2WPKH
output1_value_sat = int(float("0.15") * 100000000)
pk_hash = hash160(bytes.fromhex(alice_change_pubkey))
# Output2 is the Channel P2WSH
# The channel capacity is 5.000.000 of satoshis
output2_value_sat = int(float("0.05") * 100000000)
script_hash = hashlib.sha256(redeemScript).digest()
# The Alice UTXO was 0.201, we are leaving 0.001 as the fee of the funding trransaction

Alice pubkey is lexicographically lesser than Bob pubkey:
02245c997231079146616f70eae46dd43461b530cb55df50cac8ef321127adb963 < 032b057a643c7b928b7dc30e1f76c2a777a213fe3a7462215d10220844befe77c3
Redeem Script:
522102245c997231079146616f70eae46dd43461b530cb55df50cac8ef321127adb96321032b057a643c7b928b7dc30e1f76c2a777a213fe3a7462215d10220844befe77c352ae
channel_address:  bcrt1qhsn37ekwjgxlajuavyzne28yrahkyesgnr6qhxvzlw3807ttcjeqhcl6mc


### Creating the Unsigned Channel Funding Transaction

Now that we’ve defined all the necessary components, we can populate the fields required to construct our unsigned transaction. A transaction is considered unsigned when its witness field is left empty.

In [5]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER
marker = bytes.fromhex("00")

# FLAG
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(txid_to_spend))[::-1]
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# OUTPUT 1 to Alice Change Address
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output1_spk' 
output1_spk = bytes.fromhex("0014") + pk_hash


# OUTPUT 2 to Channel Funding Address
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output2_spk'
output2_spk = bytes.fromhex("0020") + script_hash

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
    + output2_value
    + varint_len(output2_spk)
    + output2_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_channel_funding_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)

print("unsigned_channel_funding_tx: ", unsigned_channel_funding_tx.hex())



unsigned_channel_funding_tx:  0200000001fb11bd1590ea0b4b0231f912d01b12cee195a51996088a56bc7e955385805ef80100000000ffffffff02c0e1e400000000001600145eba638eea7df425c1edc0362690b65e6216c2bf404b4c0000000000220020bc271f66ce920dfecb9d61053ca8e41f6f62660898f40b9982fba277f96bc4b200000000


We can decode this raw transaction to inspect its structure and verify that it contains all the necessary information—except for the SegWit-specific fields: the marker, the flag, and the witness data.

In [6]:
decoded = node.decoderawtransaction(unsigned_channel_funding_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

{
  "txid": "f5cab2ef35440545d4342371037a439556103692338feaac593ceac09b8649f6",
  "hash": "f5cab2ef35440545d4342371037a439556103692338feaac593ceac09b8649f6",
  "version": 2,
  "size": 125,
  "vsize": 125,
  "weight": 500,
  "locktime": 0,
  "vin": [
    {
      "txid": "f85e808553957ebc568a089619a595e1ce121bd012f931024b0bea9015bd11fb",
      "vout": 1,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": "0.15000000",
      "n": 0,
      "scriptPubKey": {
        "asm": "0 5eba638eea7df425c1edc0362690b65e6216c2bf",
        "desc": "addr(bcrt1qt6ax8rh20h6zts0dcqmzdy9kte3pds4lrzwl6s)#s9wdm0cm",
        "hex": "00145eba638eea7df425c1edc0362690b65e6216c2bf",
        "address": "bcrt1qt6ax8rh20h6zts0dcqmzdy9kte3pds4lrzwl6s",
        "type": "witness_v0_keyhash"
      }
    },
    {
      "value": "0.05000000",
      "n": 1,
      "scriptPubKey": {
        "asm": "0 bc271f66ce920dfecb9d61053ca8e41f6f626608

### Signing the Channel Funding Transaction

SegWit transactions follow a signing scheme defined in [BIP143](https://github.com/bitcoin/bips/blob/master/bip-0143.mediawiki). Depending on the sighash type (ALL, NONE, SINGLE, ANYONECANPAY), a transaction digest is produced by double-SHA256 hashing a serialized subset of the transaction, known as the tx_digest_preimage. The resulting digest is then signed and verified against a given public key.

#### Creating the Transaction Digest 

In [7]:
pk_hash = hash160(bytes.fromhex(alice_funding_pubkey))
# OP_DUP
# OP_HASH160
# OP_PUSHDATA
# PK_HASH
# OP_EQUALVERIFY
# OP_CHECKSIG

scriptcode = bytes.fromhex(
    "76"   # OP_DUP
    + "a9" # OP_HASH160
    + "14" # OP_PUSHDATA
    + pk_hash.hex()
    + "88"  # OP_EQUALVERIFY
    + "ac"  # OP_CHECKSIG
)


input_amount_sat = int(0.201 * 100_000_000)
value = input_amount_sat.to_bytes(8, byteorder="little", signed=False)

hashPrevOuts = hash256(txid + index)
hashSequence = hash256(sequence)
hashOutputs = hash256(outputs)
sighash_type = bytes.fromhex("0100 0000") # SIGHASH_ALL

tx_digest_preimage = (
    version
    + hashPrevOuts
    + hashSequence
    + txid
    + index
    + varint_len(scriptcode)
    + scriptcode
    + value
    + sequence
    + hashOutputs
    + locktime
    + sighash_type
)
print("tx_digest_preimage: ",tx_digest_preimage.hex())

tx_digest_preimage:  0200000007659a155d29e1a6012cb55a58b4d4093e64d23282b2c1ab510d91bf9136084a3bb13029ce7b1f559ef5e747fcac439f1455a2ec7c5f09b72290795e70665044fb11bd1590ea0b4b0231f912d01b12cee195a51996088a56bc7e955385805ef8010000001976a9145bd80a3647077ca7fd268dee2270a4442682ca1188aca0b3320100000000fffffffff024a453c206730b975be44da5987693c127c508aada9aabadc9455fb74b5fc60000000001000000


#### Creating the Signed Transaction

Now we’re ready to hash the transaction digest preimage and produce an ECDSA signature for it.

It’s important to note that when appending the sighash flag to the transaction preimage, we use 4 bytes (e.g., 0x01000000). However, when appending the sighash flag to the end of the signature itself, we use only 1 byte (0x01).

In this example, we’re using the most common flag: SIGHASH_ALL, which ensures that the input is only valid in a transaction with these exact inputs and outputs.

In [8]:
# Create the Transaction Digest (sigHash) to be signed
sighash = hash256(tx_digest_preimage)

# Sign the sigHash with the input private key
signing_key = ecdsa.SigningKey.from_string(bytes.fromhex(alice_funding_privkey), curve=ecdsa.SECP256k1) 
signature = signing_key.sign_digest(sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
signature = signature + bytes.fromhex("01")

# Witness field
witness = (
    # indicate the number of stack items for the txin
    # 2 items for signature and pubkey
    bytes.fromhex("02")
    + varint_len(signature)
    + signature
    + varint_len(bytes.fromhex(alice_funding_pubkey))
    + bytes.fromhex(alice_funding_pubkey)
)

# the final signed transaction
signed_channel_funding_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs # scriptsig left empty as espcified on BIP
    + output_count
    + outputs
    + witness
    + locktime
)

print("Channel Funding Signed Transaction: ",signed_channel_funding_tx.hex())

Channel Funding Signed Transaction:  02000000000101fb11bd1590ea0b4b0231f912d01b12cee195a51996088a56bc7e955385805ef80100000000ffffffff02c0e1e400000000001600145eba638eea7df425c1edc0362690b65e6216c2bf404b4c0000000000220020bc271f66ce920dfecb9d61053ca8e41f6f62660898f40b9982fba277f96bc4b202483045022100cf02c3473b90966b6aaf78a01e97579694115d518cbcf1133af5ee2a5a7faa6a02206e9d46bc22c82d6215c87353a2f659c3f3e2f41eb6e8cc3a2887f2f0746ec137012102245c997231079146616f70eae46dd43461b530cb55df50cac8ef321127adb96300000000


### The `funding_created` and `funding_signed` Messages

Now Alice can send Bob the [`funding_created`](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#the-funding_created-message) message, which includes the following information from the channel funding transaction:
- funding_txid
- funding_output_index
- signature

This message specifies the outpoint that Alice has created for the initial commitment transactions.

Bob then replies with a [`funding signed`](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#the-funding_signed-message) message, which contains:
- channel_id
- signature (Bobs signature for the First Commitment Transaction)

Important: Alice must never broadcast the channel funding transaction before receiving Bob’s signature for the first commitment transaction. Otherwise, she would have to trust Bob to cooperate in order to retrieve her funds from the multisig UTXO.

In the [first commitment transaction notebook](https://github.com/MPins/lightning-tx-tutorial/blob/master/Chapter%202%20-%20Commitment%20Transactions/First%20Commitment%20Transactions.ipynb) , we’ll go into detail on how these transactions are constructed.

Once Alice receives Bob’s signature in the `funding_signed` message, she can safely broadcast the channel funding transaction.

In [9]:
new_tx_txid = node.sendrawtransaction(signed_channel_funding_tx.hex())
# Mine a block to confirm the tx
mining_address = node.getnewaddress()
node.generatetoaddress(nblocks=1, address=mining_address, invalid_call=False)

['6f9228f30646001bbdb5d39a24ba9bdbdf0f1efd481795f5e70eb786b8e202bb']